In [152]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, utils
import matplotlib.pyplot as plt
from skimage import io, transform
from PIL import Image
import sklearn.metrics as mt
import os
from os import path
import copy
from importlib import reload
import itertools
import trainer as tr
import dataset as ds
import model as md
import predict as pr
reload(tr)
reload(ds)
reload(pr)
# %matplotlib inline


<module 'predict' from '/Users/habbes/code/class/bd4h/chexpert/src/predict.py'>

In [153]:
os.chdir('..')

In [27]:
trainer = tr.Trainer(max_train_samples=1, value_for_uncertain=0.0)

Training using options arch densenet finetune False


/Users/habbes/anaconda3/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [28]:
ts = trainer.dataloaders['train'].dataset

In [34]:
ts[2]

(tensor([[[ 0.0056,  0.0056, -0.0116,  ..., -0.3027, -0.2684, -0.2513],
          [ 0.0569,  0.0227, -0.0116,  ..., -0.3027, -0.2856, -0.2856],
          [ 0.1254,  0.0569, -0.0116,  ..., -0.3027, -0.3198, -0.3369],
          ...,
          [-1.4329, -1.4329, -1.4329,  ..., -1.0562, -1.0733, -1.1247],
          [-1.4500, -1.4500, -1.4158,  ..., -1.0562, -1.0904, -1.1418],
          [-1.4500, -1.4500, -1.4158,  ..., -1.0562, -1.1075, -1.1589]],
 
         [[ 0.1352,  0.1352,  0.1176,  ..., -0.1800, -0.1450, -0.1275],
          [ 0.1877,  0.1527,  0.1176,  ..., -0.1800, -0.1625, -0.1625],
          [ 0.2577,  0.1877,  0.1176,  ..., -0.1800, -0.1975, -0.2150],
          ...,
          [-1.3354, -1.3354, -1.3354,  ..., -0.9503, -0.9678, -1.0203],
          [-1.3529, -1.3529, -1.3179,  ..., -0.9503, -0.9853, -1.0378],
          [-1.3529, -1.3529, -1.3179,  ..., -0.9503, -1.0028, -1.0553]],
 
         [[ 0.3568,  0.3568,  0.3393,  ...,  0.0431,  0.0779,  0.0953],
          [ 0.4091,  0.3742,

In [30]:
trainer2 = tr.Trainer(max_train_samples=1, value_for_uncertain=1.0)
ts2 = trainer2.dataloaders['train'].dataset

Training using options arch densenet finetune False


/Users/habbes/anaconda3/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [35]:
ts2[2]

(tensor([[[-2.0323, -1.6042, -0.9363,  ...,  2.0092,  1.9920,  1.9920],
          [-2.0323, -1.5870, -0.9020,  ...,  2.0263,  2.0092,  2.0092],
          [-2.0494, -1.5528, -0.8164,  ...,  2.0434,  2.0434,  2.0434],
          ...,
          [-0.9363, -0.9877, -1.0733,  ..., -0.8849, -0.9534, -0.9877],
          [-1.0390, -1.0733, -1.1075,  ..., -0.8678, -0.9192, -0.9363],
          [-1.0904, -1.1075, -1.1247,  ..., -0.8678, -0.9020, -0.9192]],
 
         [[-1.9482, -1.5105, -0.8277,  ...,  2.1835,  2.1660,  2.1660],
          [-1.9482, -1.4930, -0.7927,  ...,  2.2010,  2.1835,  2.1835],
          [-1.9657, -1.4580, -0.7052,  ...,  2.2185,  2.2185,  2.2185],
          ...,
          [-0.8277, -0.8803, -0.9678,  ..., -0.7752, -0.8452, -0.8803],
          [-0.9328, -0.9678, -1.0028,  ..., -0.7577, -0.8102, -0.8277],
          [-0.9853, -1.0028, -1.0203,  ..., -0.7577, -0.7927, -0.8102]],
 
         [[-1.7173, -1.2816, -0.6018,  ...,  2.3960,  2.3786,  2.3786],
          [-1.7173, -1.2641,

In [36]:
tdf = pd.read_csv(ds.TRAIN_CSV)

In [37]:
vdf = pd.read_csv(ds.VALID_CSV)

In [46]:
tdf['Path'][200]

'CheXpert-v1.0-small/train/patient00063/study5/view1_frontal.jpg'

In [42]:
vdf.head()['Path'][100]

KeyError: 100

In [76]:
d = tdf.iloc[[0, 1]][['Cardiomegaly', 'Edema', 'Consolidation', 'Pneumonia']]
d

,Cardiomegaly,Edema,Consolidation,Pneumonia
0,NaN,NaN,NaN,NaN
1,-1.0,-1.0,-1.0,NaN


In [77]:
d[['Cardiomegaly', 'Edema']] = d[['Cardiomegaly', 'Edema']].replace(-1, 0).replace(np.NaN, 0)
d[['Consolidation', 'Pneumonia']] = d[['Consolidation', 'Pneumonia']].replace(-1, 1).replace(np.NaN, 0)
d

,Cardiomegaly,Edema,Consolidation,Pneumonia
0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0


In [78]:
d.values

array([[0., 0., 0., 0.],
       [0., 0., 1., 0.]])

In [86]:
tdf['side'] = tdf['Path'].apply(lambda p: 'lateral' if 'lateral' in p else 'frontal')
tdf['name'] = tdf['Path'].apply(lambda p: os.path.basename(p))
tdf[['Path', 'name', 'side']]

,Path,name,side
0,CheXpert-v1.0-small/train/patient00001/study1/...,view1_frontal.jpg,frontal
1,CheXpert-v1.0-small/train/patient00002/study2/...,view1_frontal.jpg,frontal
2,CheXpert-v1.0-small/train/patient00002/study1/...,view1_frontal.jpg,frontal
3,CheXpert-v1.0-small/train/patient00002/study1/...,view2_lateral.jpg,lateral
4,CheXpert-v1.0-small/train/patient00003/study1/...,view1_frontal.jpg,frontal
5,CheXpert-v1.0-small/train/patient00004/study1/...,view1_frontal.jpg,frontal
6,CheXpert-v1.0-small/train/patient00004/study1/...,view2_lateral.jpg,lateral
7,CheXpert-v1.0-small/train/patient00005/study1/...,view1_frontal.jpg,frontal
8,CheXpert-v1.0-small/train/patient00005/study1/...,view2_lateral.jpg,lateral
9,CheXpert-v1.0-small/train/patient00005/study2/...,view1_frontal.jpg,frontal


In [81]:
'catd' in 'sdffcatsdf'

False

In [97]:
side='lateral'
tdf.query(f"side=='{side}'").shape

(32387, 21)

In [98]:
vdf['side'] = vdf['Path'].apply(lambda p: 'lateral' if 'lateral' in p else 'frontal')
vdf[['Path', 'side']]

,Path,side
0,CheXpert-v1.0-small/valid/patient64541/study1/...,frontal
1,CheXpert-v1.0-small/valid/patient64542/study1/...,frontal
2,CheXpert-v1.0-small/valid/patient64542/study1/...,lateral
3,CheXpert-v1.0-small/valid/patient64543/study1/...,frontal
4,CheXpert-v1.0-small/valid/patient64544/study1/...,frontal
5,CheXpert-v1.0-small/valid/patient64545/study1/...,frontal
6,CheXpert-v1.0-small/valid/patient64546/study1/...,frontal
7,CheXpert-v1.0-small/valid/patient64547/study1/...,frontal
8,CheXpert-v1.0-small/valid/patient64547/study1/...,frontal
9,CheXpert-v1.0-small/valid/patient64547/study1/...,lateral


In [104]:
vdf.query("side=='lateral'")[ds.LABELS]

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion
2,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0
14,0.0,0.0,0.0,0.0,0.0
20,0.0,0.0,0.0,0.0,0.0
23,0.0,0.0,0.0,0.0,0.0
25,0.0,0.0,0.0,0.0,0.0
27,1.0,0.0,0.0,0.0,0.0
34,1.0,0.0,0.0,0.0,0.0
36,0.0,0.0,0.0,0.0,0.0
40,0.0,0.0,0.0,0.0,0.0


In [138]:
os.chdir('src')
reload(ds)
os.chdir('..')

In [119]:
l = ds.get_val_loader(side='lateral')
l.dataset.df.shape

(32, 20)

In [128]:
class TM(nn.Module):
    def __init__(self):
        super(TM, self).__init__()
        self.add_module('mod', nn.Linear(20, 5))
    def forward(self, x, i):
        print("x", x, "i", i)
        return self.mod(x)

In [129]:
tm = TM()

In [131]:
tm(torch.rand(20), 1)

x tensor([0.4380, 0.7955, 0.5820, 0.3397, 0.2745, 0.7992, 0.4577, 0.3406, 0.3058,
        0.6810, 0.7074, 0.9637, 0.8867, 0.6979, 0.0147, 0.8817, 0.0390, 0.9885,
        0.0704, 0.5810]) i 1


tensor([-0.0416,  0.5722, -0.4439, -0.5907,  0.0830], grad_fn=<AddBackward0>)

In [140]:
l = ds.get_val_loader_for_multiside()
l

In [144]:
a, b, c = next(iter(l))
a[0], b[0], c[0]

(tensor([[[-1.8097, -1.8097, -1.8268,  ..., -1.6727, -1.7412, -1.7925],
          [-1.8268, -1.8268, -1.8268,  ..., -1.4329, -1.5185, -1.6384],
          [-1.7925, -1.7925, -1.8097,  ..., -1.3644, -1.3815, -1.4158],
          ...,
          [ 1.8037,  1.7694,  1.7865,  ...,  1.9749,  1.9749,  1.9749],
          [ 1.8037,  1.8037,  1.8037,  ...,  1.9749,  2.0092,  2.0092],
          [ 1.8208,  1.8037,  1.8379,  ...,  2.0092,  2.0263,  1.9920]],
 
         [[-1.7206, -1.7206, -1.7381,  ..., -1.5805, -1.6506, -1.7031],
          [-1.7381, -1.7381, -1.7381,  ..., -1.3354, -1.4230, -1.5455],
          [-1.7031, -1.7031, -1.7206,  ..., -1.2654, -1.2829, -1.3179],
          ...,
          [ 1.9734,  1.9384,  1.9559,  ...,  2.1485,  2.1485,  2.1485],
          [ 1.9734,  1.9734,  1.9734,  ...,  2.1485,  2.1835,  2.1835],
          [ 1.9909,  1.9734,  2.0084,  ...,  2.1835,  2.2010,  2.1660]],
 
         [[-1.4907, -1.4907, -1.5081,  ..., -1.3513, -1.4210, -1.4733],
          [-1.5081, -1.5081,

In [147]:
a[0].to(torch.device('cpu'))

tensor([[[-1.8097, -1.8097, -1.8268,  ..., -1.6727, -1.7412, -1.7925],
         [-1.8268, -1.8268, -1.8268,  ..., -1.4329, -1.5185, -1.6384],
         [-1.7925, -1.7925, -1.8097,  ..., -1.3644, -1.3815, -1.4158],
         ...,
         [ 1.8037,  1.7694,  1.7865,  ...,  1.9749,  1.9749,  1.9749],
         [ 1.8037,  1.8037,  1.8037,  ...,  1.9749,  2.0092,  2.0092],
         [ 1.8208,  1.8037,  1.8379,  ...,  2.0092,  2.0263,  1.9920]],

        [[-1.7206, -1.7206, -1.7381,  ..., -1.5805, -1.6506, -1.7031],
         [-1.7381, -1.7381, -1.7381,  ..., -1.3354, -1.4230, -1.5455],
         [-1.7031, -1.7031, -1.7206,  ..., -1.2654, -1.2829, -1.3179],
         ...,
         [ 1.9734,  1.9384,  1.9559,  ...,  2.1485,  2.1485,  2.1485],
         [ 1.9734,  1.9734,  1.9734,  ...,  2.1485,  2.1835,  2.1835],
         [ 1.9909,  1.9734,  2.0084,  ...,  2.1835,  2.2010,  2.1660]],

        [[-1.4907, -1.4907, -1.5081,  ..., -1.3513, -1.4210, -1.4733],
         [-1.5081, -1.5081, -1.5081,  ..., -1

In [171]:
os.chdir('src')
reload(md)
os.chdir('..')
ws = torch.load('models/multiside/frontal/checkpoint_0.pth')
f = md.get_model(pretrained=True, arch='resnet')
f.load_state_dict(ws)
l = md.get_model(pretrained=True, arch='resnet')
l.load_state_dict(torch.load('models/multiside/lateral/checkpoint_0.pth'))
multi = md.MultiSide(f, l)

In [172]:
multi(a, c)

image_path ('./data/CheXpert-v1.0-small/valid/patient64541/study1/view1_frontal.jpg',) ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.

tensor([[-0.3840, -0.1487, -0.2930, -0.5885,  0.8455]],
       grad_fn=<AddmmBackward>)